In [1]:
#Import dependencies
import pandas as pd
import numpy as np
import 

In [ ]:
# Narrative/intro goes here

In [2]:
# Stacy start

In [ ]:
# Plot the naturalization data

## CLEAN THIS UP TO MATCH CORRECT VARIABLES

plt.plot(naturalization.index, naturalization[['Petitions filed', 'Naturalized, total', 'Petitions denied']])
plt.tight_layout()

# Spike in 1996 - and general increase - can be explained by this bill: https://www.vox.com/2016/4/28/11515132/iirira-clinton-immigration
# Need to plot just the 2000 - 2010, then 2015 - 2018 data - to examine whether there are trends

In [3]:
# Stacy end

In [4]:
# Kana start

In [5]:
# Kana end

In [6]:
# Satish start

In [7]:
# Satish end

In [8]:
# Umar start
#Read in the Excel file and view the headers
Lawful_df = pd.read_excel("./Resources/fy2018_Lawful.xlsx", header=4)
Lawful_df.head()

In [ ]:
#Use the iloc function to locate the point of interest in a data set
Broad_age = Lawful_df.iloc[19:23,:]
Broad_age

In [ ]:
#View the characteristic and Total columns 
Broad_age_df = pd.DataFrame(Broad_age)
Sex = Broad_age_df.iloc[:, 0:2]
Sex

In [ ]:
#Rename your columns 
Cleaned = Sex.rename(columns={"Characteristic": "Age", "Total": "Lawful 2018"})
Index_age = Cleaned.set_index("Age")
Index_age

In [ ]:
#View a single row 
Sex = Lawful_df.iloc[19,1:]
Sex.to_frame(name="LPR 2018")

In [ ]:
#Use the iloc function to locate the point of interest in a data set
Marital_status = Lawful_df.iloc[24:30,:]
Marital_status

In [ ]:
#View the characteristic and Total columns 
New_marital_df = pd.DataFrame(Marital_status)
Specific = New_marital_df.iloc[:, 0:2]
Specific

In [ ]:
#Rename your columns 
Renamed = Specific.rename(columns={"Characteristic": "Marital Status", "Total": "Lawful Permanent Resident 2018"})
Renamed.head()

In [ ]:
#View a single column 
Status = Lawful_df.iloc[24:30,0:2]
Name18 = Status.rename(columns={"Characteristic": "Marital Status", "Total": "Lawful 2018"})
#Set marital status as index for clarity
Name18.set_index("Marital Status", inplace=True)
Name18

In [ ]:
#Read in the Excel file and 
Lawful09_df = pd.read_excel("./Resources/fy2009_Lawful.xls", header=4)
Lawful09_df.head()

In [ ]:
#Use the iloc function to locate the point of interest in a data set
Broad09_age = Lawful09_df.iloc[20:24,:]
Broad09_age

In [ ]:
#View the characteristic and Total columns 
Broad09_age_df = pd.DataFrame(Broad09_age)
Sex09 = Broad09_age_df.iloc[:, 0:2]
Sex09

In [ ]:
#Rename your columns 
Age09 = Sex09.rename(columns={"Characteristic": "Age", "Total": "Lawful Permanent Resident 2009"})
New_index09 = Age09.set_index("Age")
New_index09

In [ ]:
#View a single row 
Sex09 = Lawful09_df.iloc[20,1:]
Sex09.to_frame(name="LPR 2009")

In [ ]:
#Use the iloc function to locate the point of interest in a data set
Marital09_status = Lawful09_df.iloc[26:32,:]
Marital09_status

In [ ]:
#View the characteristic and Total columns 
New09_marital_df = pd.DataFrame(Marital09_status)
Specific09 = New09_marital_df.iloc[:, 0:2]
Specific09

In [ ]:
#Rename your columns 
Renamed09 = Specific09.rename(columns={"Characteristic": "Marital Status", "Total": "Lawful Permanent Resident 2009"})
Renamed09.head()

In [ ]:
#View a single column
Status09 = Lawful09_df.iloc[26:32,0:2]
Name09 = Status09.rename(columns={"Characteristic": "Marital Status", "Total": "Lawful 2009"})
#Set marital status as index for clarity
Name09.set_index("Marital Status", inplace=True)
Name09

#Umar Code ends here

In [ ]:
# Narrative/summary goes here